In [23]:
import selenium
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import time
import pandas as pd

### Naver뉴스에서 크롤링할 뉴스 카테고리 선택

In [28]:
category = 4 # 1: 정치, 2: 경제, 3: 사회, 4: 생활/문화, 5: IT/과학, 6:세계

### 네이버 뉴스 페이지 접속

In [29]:
URL = "https://news.naver.com/"
driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)

elements = driver.find_elements_by_class_name("Nlist_item")
elements[category].click()

time.sleep(1)

### 뉴스기사 크롤링

In [30]:
##### 초기화 #####

headlines = []
contents = []
urls = []
dates = []

##### 1페이지에 있는 뉴스기사 크롤링 #####

body = driver.find_element_by_class_name("section_body")
links = body.find_elements_by_class_name("photo")
for i in range(len(links)):
    target = links[i].find_element_by_tag_name("a")
    target.send_keys(Keys.CONTROL + "\n")
time.sleep(1)
for i in range(len(links)):
    driver.switch_to_window(driver.window_handles[len(driver.window_handles)-1])
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "media_end_head_headline")))
    except:
        driver.close()
        continue
    headlines.append(driver.find_element_by_class_name("media_end_head_headline").text)
    contents.append(driver.find_element_by_id("dic_area").text)
    dates.append(driver.find_element_by_class_name("media_end_head_info_datestamp").find_element_by_tag_name("span").text)
    urls.append(driver.current_url)
    driver.close()

driver.switch_to_window(driver.window_handles[0])

##### 2페이지부터 모든 페이지까지 뉴스기사 크롤링 #####

paging = driver.find_element_by_class_name("paging")
pages = paging.find_elements_by_tag_name("a")
while(1):
    for page in pages:
        if(page.text != "이전"):
            page.send_keys(Keys.CONTROL + "\n")
            time.sleep(1)
            driver.switch_to_window(driver.window_handles[-1])
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "section_body")))
            body = driver.find_element_by_class_name("section_body")
            links = body.find_elements_by_class_name("photo")

            for i in range(len(links)):
                target = links[i].find_element_by_tag_name("a")
                target.send_keys(Keys.CONTROL + "\n")
                
            time.sleep(1)
            for i in range(len(links)):
                driver.switch_to_window(driver.window_handles[len(driver.window_handles)-1])
                try:
                    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "media_end_head_headline")))
                except:
                    driver.close()
                    continue
                headlines.append(driver.find_element_by_class_name("media_end_head_headline").text)
                contents.append(driver.find_element_by_id("dic_area").text)
                dates.append(driver.find_element_by_class_name("media_end_head_info_datestamp").find_element_by_tag_name("span").text)
                urls.append(driver.current_url)
                driver.close()
            driver.switch_to_window(driver.window_handles[-1])
            driver.close()
            driver.switch_to_window(driver.window_handles[0])
    if(pages[-1].text != "다음"):
        break
    
    pages[-1].send_keys(Keys.CONTROL + "\n")
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "paging")))
    paging = driver.find_element_by_class_name("paging")
    pages = paging.find_elements_by_tag_name("a")
    time.sleep(3)

driver.close() # Chrome Webdriver 종료

<ipython-input-30-45466ad1c283>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[len(driver.window_handles)-1])
<ipython-input-30-45466ad1c283>:29: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])
<ipython-input-30-45466ad1c283>:40: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-30-45466ad1c283>:51: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[len(driver.window_handles)-1])
<ipython-input-30-45466ad1c283>:63: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-30-45466ad1c283>:65: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])
<ipython-input-30-45466ad1c283>:71: DeprecationWarning: use driver.switch_to.win

Error


### 크롤링한 데이터를 csv파일로 저장

In [31]:
df = pd.DataFrame()

df['title'] = headlines
df['text'] = contents
df['url'] = urls
df['date'] = dates
df = df.drop_duplicates(ignore_index = True)

categories = ['', 'politics', 'economy', 'society', 'life', 'IT', 'world']
df.to_csv("NaverNews_" + categories[category] + ".csv", encoding = 'utf-8-sig', index = False)